In [1]:
import time
import nest_asyncio
import requests
import json
from urllib import request
import multiprocessing
import asyncio
import aiohttp
from concurrent.futures import ThreadPoolExecutor

#### Synchronicznie 

In [3]:
def download_images():
    response = requests.get("https://picsum.photos/v2/list")
    if response.status_code != 200:
        raise AttributeError('GET /tasks/ {}'.format(response.status_code))
    data = json.loads(response.text)[:10]

    pictures=[]
    for s in data:
        pictures.append(s['download_url']+".jpg")
    return pictures

def saveImages(link):
    filename = link.split('/')[6].split('.')[0]
    fileformat = link.split('/')[6].split('.')[1]
    request.urlretrieve(link, "{}.{}".format(filename, fileformat))

def main():
    images = download_images()
    for image in images:
        saveImages(image)

start_time = time.time()
main()
synch_duration = time.time() - start_time
print(f"Time taken: {synch_duration}")

Time taken: 5.076997518539429


#### Wielowątkowo

In [4]:
def process_images_threading():
    images = download_images()
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(saveImages,images)

start_time = time.time()
process_images_threading()
mth_duration = time.time() - start_time
print(f"Time taken: {mth_duration}")

Time taken: 1.9130001068115234


#### AsyncIO

In [5]:
async def download_images_asyncio(link, session):
    filename = link.split('/')[6].split('.')[0]
    fileformat = link.split('/')[6].split('.')[1]
    async with session.get(link) as response:
        with open("{}.{}".format(filename, fileformat), 'wb') as fd:
            async for data in response.content.iter_chunked(1024):
                fd.write(data)

async def main_asyncio():
    images = download_images()

    async with aiohttp.ClientSession() as session:
        tasks=[download_images_asyncio(image,session)for image in images]
        return await asyncio.gather(*tasks)

start_time = time.time()
nest_asyncio.apply()
asyncio.run(main_asyncio())
asyncio_duration = time.time() - start_time
print(f"Time taken: {asyncio_duration}")

Time taken: 1.4619972705841064


#### Porównanie: 

In [6]:
print(f"Synchronicznie: {synch_duration} sekund")
print(f"Wielowątkowo: {mth_duration} sekund")
print(f"AsyncIO: {asyncio_duration} sekund")

Synchronicznie: 5.076997518539429 sekund
Wielowątkowo: 1.9130001068115234 sekund
AsyncIO: 1.4619972705841064 sekund
